This is the original preprocessing of semeval 2010 data. Just tokenization is performed. No stop word removal, no punctuation removal, no entity blinding

In [2]:
%load_ext autoreload

In [127]:
%autoreload

import os, random, pandas as pd, numpy as np
import pickle
import sys
import ast
sys.path.append('../../')
# sys.path.append('../ddi_preprocess')
from relation_extraction.data import utils
import nltk
from ast import literal_eval
import itertools
from ast import literal_eval # to convert the string tuple form to an actual tuple
RESOURCE_PATH = "/data/medg/misc/geeticka/relation_extraction/semeval_2010"
outdir = 'pre-processed/original/'
def res(path): return os.path.join(RESOURCE_PATH, path)
from relation_extraction.data.converters.converter_semeval2010 import get_dataset_dataframe, write_dataframe,\
read_dataframe, check_equality_of_written_and_read_df, rev_relation_dict, write_into_txt

In [128]:
df_train = get_dataset_dataframe(res('TRAIN_FILE.TXT'))

In [129]:
df_test = get_dataset_dataframe(res('TEST_FILE_FULL.TXT'))

In [130]:
len(df_train)

8000

In [131]:
len(df_test)

2717

The lengths seem to be correct. Now we want to check if any entity is empty/ was not detected

## Testing for empty entities

In [132]:
def get_empty_entity_rows(df):
    empty_entity_rows = []
    def find_empty_entity_number(row):
        metadata = row.metadata
        e1 = metadata['e1']['word_index']
        e2 = metadata['e2']['word_index']
        if not e1 or not e2:
            empty_entity_rows.append(row.row_num)
    temp_df = df.copy()
    temp_df.insert(0, 'row_num', range(0, len(temp_df)))
    temp_df.apply(find_empty_entity_number, axis=1)
    return empty_entity_rows

def get_empty_rows_array(empty_entity_rows, df):
    empty_rows_array = []
    for index in empty_entity_rows:
        e1 = df.iloc[index].e1
        e2 = df.iloc[index].e2
        original_sentence = df.iloc[index].original_sentence
        tokenized_sentence = df.iloc[index].tokenized_sentence
        metadata = df.iloc[index].metadata
        empty_rows_array.append([index, original_sentence, e1, e2, metadata, tokenized_sentence])
    new_df = pd.DataFrame(data=empty_rows_array,    # values
             columns=['index_original', 'original_sentence' , 'e1', 'e2', 'metadata', 'tokenized_sentence'])
    return empty_rows_array, new_df

In [133]:
def get_empty_vals(df):
    empty_entity_rows = get_empty_entity_rows(df)
    empty_rows_array, new_df = get_empty_rows_array(empty_entity_rows, df)
    return empty_rows_array, new_df

In [134]:
get_empty_vals(df_train)

([], Empty DataFrame
 Columns: [index_original, original_sentence, e1, e2, metadata, tokenized_sentence]
 Index: [])

In [135]:
get_empty_vals(df_test)

([], Empty DataFrame
 Columns: [index_original, original_sentence, e1, e2, metadata, tokenized_sentence]
 Index: [])

## Write into the csv format

In [148]:
if not os.path.exists(res(outdir)):
    os.makedirs(res(outdir))

In [149]:
write_dataframe(df_train, res(outdir + 'train_original.csv'))

In [150]:
df_train_copy = read_dataframe(res(outdir + 'train_original.csv'))

In [151]:
# The first checks with the pd.equals method, and the other does a manual checking per column
check_equality_of_written_and_read_df(df_train, df_train_copy)

(True, True)

In [152]:
write_dataframe(df_test, res(outdir + 'test_original.csv'))

In [153]:
df_test_copy = read_dataframe(res(outdir + 'test_original.csv'))

In [154]:
check_equality_of_written_and_read_df(df_test, df_test_copy)

(True, True)

## Write the files into txt format

In [156]:
write_into_txt(df_train, res(outdir + 'train_original.txt'))


Unique relations: 	 ['Component-Whole(e2,e1)' 'Other' 'Instrument-Agency(e2,e1)'
 'Member-Collection(e1,e2)' 'Cause-Effect(e2,e1)'
 'Entity-Destination(e1,e2)' 'Content-Container(e1,e2)'
 'Message-Topic(e1,e2)' 'Product-Producer(e2,e1)'
 'Member-Collection(e2,e1)' 'Entity-Origin(e1,e2)' 'Cause-Effect(e1,e2)'
 'Component-Whole(e1,e2)' 'Message-Topic(e2,e1)' 'Product-Producer(e1,e2)'
 'Entity-Origin(e2,e1)' 'Content-Container(e2,e1)'
 'Instrument-Agency(e1,e2)' 'Entity-Destination(e2,e1)']


In [157]:
write_into_txt(df_test, res(outdir + 'test_original.txt'))

Unique relations: 	 ['Message-Topic(e1,e2)' 'Product-Producer(e2,e1)'
 'Instrument-Agency(e2,e1)' 'Entity-Destination(e1,e2)'
 'Cause-Effect(e2,e1)' 'Component-Whole(e1,e2)' 'Product-Producer(e1,e2)'
 'Member-Collection(e2,e1)' 'Other' 'Entity-Origin(e1,e2)'
 'Content-Container(e1,e2)' 'Entity-Origin(e2,e1)' 'Cause-Effect(e1,e2)'
 'Component-Whole(e2,e1)' 'Content-Container(e2,e1)'
 'Instrument-Agency(e1,e2)' 'Message-Topic(e2,e1)'
 'Member-Collection(e1,e2)' 'Entity-Destination(e2,e1)']
